In [1]:
#!pip install tweepy

In [2]:
import tweepy
import json
import csv
from datetime import date
from datetime import datetime
import time

In [3]:
path = './data/'

In [4]:
from secrets import *

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [6]:
# Helper function (save data in json file)
def save_json(file_name, file_content):
    with open(path + file_name, 'w', encoding='utf-8') as f:
        json.dump(file_content, f, ensure_ascii=False, indent=4)

In [7]:
# Helper function (rate limit)
# Helper function to handle twitter API rate limit
def limit_handled(cursor, list_name):
    while True:
        try:
            yield cursor.next()
        # Catch Twitter API rate limit exception and wait for 15 minutes
        except tweepy.RateLimitError:
            print("Data points in list = {}".format(len(list_name)))
            print("Hit Twitter API rate limit.")
            for i in range(3, 0, -1):
                print("Wait for {} mins.".format(i*5))
                time.sleep(5*60)
        # Catch any other Twitter API exceptions
        except tweepy.error.TweepError:
            print('\nCaught TweepError exception')
                

In [8]:
# get friend list (also known as following list)

def get_friends():
    
    # Create a list to store friends data
    friends_list = []
    # For-loop to iterate over tweepy cursors
    cursors = tweepy.Cursor(api.friends, count=200).pages()
    for i, page in enumerate(limit_handled(cursor, friends_list)):
        print("\r" + "Loading"+ i % 5 * ".", end='')
        
         # Add latest batch of friends information
        friends_list += page
        
    # Add latest batch of friend data to the list
    friends_list = [x._json for x in friends_list]
    # Save the data in a JSON file
    save_json('friends_data.json', friends_list)
            

In [9]:
# Helper function to get all tweets of a specific user
# NOTE: This method only allows access to the most recent 3200 tweets
# Source: https://gist.github.com/yanofsky/5436496

def get_all_tweets(screen_name):
    # initialize a list to hold all the Tweets
    alltweets = []
    # make initial request for most recent tweets
    # (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name, count=200)
    # save most recent tweets
    alltweets.extend(new_tweets)
    # save the id of the oldest tweet less one to avoid duplication
    oldest = alltweets[-1].id - 1
    # keep grabbing tweets until there are no tweets left
    while len(new_tweets) > 0:
        print("getting tweets before %s" %(oldest))
        # all subsequent requests use max_id param to prevent
        # duplicate
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        # save most recent tweets
        alltweets.extend(new_tweets)
        # update the id of the oldest tweet less one
        oldest = alltweets[-1].id = 1
        print("...%s tweets downloaded so far" %(len(alltweets)))
        ### END OF WHILE LOOP ###

    # transform the tweepy tweets into a 2D array that will
    # populate the csv
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text, tweet.favorite_count,tweet.in_reply_to_screen_name, tweet.retweeted] for tweet in alltweets]
    with open(path + '%s_tweets.csv' % screen_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["id", "created_at", "text", "likes", "in reply to", "retweeted"])
        writer.writerows(outtweets)
    pass

In [10]:
get_all_tweets ('the_airnomad')

getting tweets before 1293924138353008643
...38 tweets downloaded so far
